<a href="https://colab.research.google.com/github/dasari2023/Code/blob/main/EfficientNetIDRiD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import image_data_format
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from keras.models import Sequential
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from keras.layers import Dense
from keras.layers import Conv2D, DepthwiseConv2D, SeparableConv2D
from keras.layers import AvgPool2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
#from keras.constraints import maxnorm
import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import sys
import glob
import keras
import cv2
import csv
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

In [ ]:
pip install pytesseract

In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import image_to_string

In [ ]:
pip install opencv-python

In [ ]:
x,y1, y = [], [],[]

x_train=list()
y_train=list()

dgtrain =glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Training/*.jpg')
ltrain =glob.glob('/content/drive/MyDrive/Dataset/Localization/Training/*.jpg')
strain =glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Training/*.jpg')
for i in dgtrain:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(224,224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(0)
for j in ltrain:
    img=cv2.imread(j,1)
    img=cv2.resize(img,(224,224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(1)
for k in strain:
    img=cv2.imread(k,1)
    img=cv2.resize(img,(224,224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(2)

In [ ]:

dgtest = glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Testing/*.jpg')

ltest =glob.glob('/content/drive/MyDrive/Dataset/Localization/Testing/*.jpg')
stest =glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Testing/*.jpg')
x_test=list()
y_test=list()
for i in dgtest:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(224, 224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(0)
for j in ltest:
    img=cv2.imread(j,1)
    img=cv2.resize(img,(224, 224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(1)
for k in stest:
    img=cv2.imread(k,1)
    img=cv2.resize(img,(224, 224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(2)

In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your dataset paths
dgtrain = glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Training/*.jpg')
ltrain = glob.glob('/content/drive/MyDrive/Dataset/Localization/Training/*.jpg')
strain = glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Training/*.jpg')

dgtest = glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Testing/*.jpg')
ltest = glob.glob('/content/drive/MyDrive/Dataset/Localization/Testing/*.jpg')
stest = glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Testing/*.jpg')

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path, 1)
    img = cv2.resize(img, (224, 224))
    img = np.float32(img) / 255.0
    img = img.reshape(224, 224, 3)
    return img

# Load and preprocess training data
x = []
y = []

for i in dgtrain:
    img = load_and_preprocess_image(i)
    x.append(img)
    y.append(0)

for j in ltrain:
    img = load_and_preprocess_image(j)
    x.append(img)
    y.append(1)

for k in strain:
    img = load_and_preprocess_image(k)
    x.append(img)
    y.append(2)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Create an ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate augmented images and append to the training set
for i in range(len(x_train)):
    img = x_train[i]
    img = img.reshape((1,) + img.shape)  # Reshape to (1, height, width, channels) for flow method
    for batch in datagen.flow(img, batch_size=1):
        x_train.append(batch[0])
        y_train.append(y_train[i])
        break  # Exit the loop after one batch to avoid infinite loop

# Similarly, you can perform data augmentation for validation data if needed

# Convert lists to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_val)
y_val = np.array(y_val)

# Now, x_train and y_train contain augmented training data
# x_val and y_val contain validation data
# You can use these arrays to train your model


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming you have loaded your data into the variables x and y correctly
# x should contain the images, and y should contain the labels.

# Convert data to NumPy arrays
x_samp = np.asarray(x)
y_samp = np.asarray(y)

# Check the shapes of your data to ensure they match
print("x_samp.shape:", x_samp.shape)
print("y_samp.shape:", y_samp.shape)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_samp, y_samp, test_size=0.2, random_state=42)

# Check the shapes of the split datasets
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
#print("x_test shape:", x_test.shape)
#print("y_test shape:", y_test.shape)

x_samp.shape: (880, 224, 224, 3)
y_samp.shape: (880,)
x_train shape: (704, 224, 224, 3)
y_train shape: (704,)


In [ ]:
pip install keras scikit-learn

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.utils import to_categorical

# Assuming x_train, y_train, x_test, y_test are defined from the previous code

# Convert labels to one-hot encoding
num_classes = 3  # Replace with the actual number of classes in your dataset
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

# Load EfficientNetB7 pre-trained on ImageNet
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build a model on top of the pre-trained base model
model = Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

# Freeze the weights of the pre-trained base model
base_model.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
evaluation = model.evaluate(x_test, y_test_one_hot, batch_size=32)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

258076736/258076736 [==============================] - 5s 0us/step
Epoch 1/10
18/18 [==============================] - 294s 15s/step - loss: 1.0537 - accuracy: 0.4529 - val_loss: 0.9402 - val_accuracy: 0.4894
Epoch 2/10
18/18 [==============================] - 282s 16s/step - loss: 0.9618 - accuracy: 0.4831 - val_loss: 0.8428 - val_accuracy: 0.4894
Epoch 3/10
18/18 [==============================] - 273s 15s/step - loss: 0.9378 - accuracy: 0.4600 - val_loss: 0.8454 - val_accuracy: 0.4894
Epoch 4/10
18/18 [==============================] - 273s 15s/step - loss: 0.9258 - accuracy: 0.4458 - val_loss: 0.8624 - val_accuracy: 0.4894
Epoch 5/10
18/18 [==============================] - 266s 15s/step - loss: 0.9422 - accuracy: 0.4583 - val_loss: 0.8473 - val_accuracy: 0.4681
Epoch 6/10
18/18 [==============================] - 268s 15s/step - loss: 0.9140 - accuracy: 0.4352 - val_loss: 0.8467 - val_accuracy: 0.4681
Epoch 7/10
18/18 [==============================] - 267s 15s/step - loss: 0.9257 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import numpy as np
import copy
import random
import csv

# Assuming x and y are your data and labels
x = np.array(x)
y = np.array(y)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
NUM_OF_CLASSES = 3  # Replace with the actual number of classes in your dataset
y_train_one_hot = to_categorical(y_train, num_classes=NUM_OF_CLASSES)
y_test_one_hot = to_categorical(y_test, num_classes=NUM_OF_CLASSES)

# Model configuration
NUM_OF_CLIENTS = 3
SELECT_CLIENTS = 0.5
EPOCHS = 4
CLIENT_EPOCHS = 5
BATCH_SIZE = 10
DROP_RATE = 0
learning_rate = 0.0025

# Define serverhist dictionary to store evaluation metrics
serverhist = {
    'loss': [],
    'accuracy': [],
    'f1': [],
    'precision': [],
    'recall': [],
    'specificity': [],
    'sensitivity': []
}

# Load EfficientNetB7 pre-trained on ImageNet
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build a model on top of the pre-trained base model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(NUM_OF_CLASSES, activation='softmax'))

# Freeze the weights of the pre-trained base model
base_model.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.1)

# Split the data for each client
x_train_clients = np.array_split(x_train, NUM_OF_CLIENTS)
y_train_clients = np.array_split(y_train_one_hot, NUM_OF_CLIENTS)

# Initialize avg_weight
avg_weight = model.get_weights()

# Federated Learning Loop
for epoch in range(EPOCHS):
    client_models = [Sequential() for _ in range(NUM_OF_CLIENTS)]

    server_weight = []
    selected_num = int(max(NUM_OF_CLIENTS * SELECT_CLIENTS, 1))
    split_data_index = random.sample(range(NUM_OF_CLIENTS), selected_num)

    for index in split_data_index:
        client_models[index].add(base_model)
        client_models[index].add(Flatten())
        client_models[index].add(Dense(256, activation='relu'))
        client_models[index].add(Dense(NUM_OF_CLASSES, activation='softmax'))

        client_models[index].compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

        # Client update
        client_models[index].set_weights(avg_weight)
        client_models[index].fit(x_train_clients[index], y_train_clients[index],
                                 epochs=CLIENT_EPOCHS,
                                 batch_size=BATCH_SIZE,
                                 verbose=1,
                                 validation_split=0.2)

        server_weight.append(copy.deepcopy(client_models[index].get_weights()))

    avg_weight = np.mean(server_weight, axis=0)
    model.set_weights(avg_weight)

    # Evaluate the model on the test set
    evaluation = model.evaluate(x_test, y_test_one_hot, batch_size=32)

    serverhist['accuracy'].append(evaluation[1])
    serverhist['loss'].append(evaluation[0])

# Print the results
print("\t\t\tFed Avg")
print("Round\tAccuracy\tLoss")
for i in range(EPOCHS):
    print("{}\t{}\t{}".format(
        i + 1,
        serverhist['accuracy'][i],
        serverhist['loss'][i]
    ))

Epoch 1/10
20/20 [==============================] - 283s 13s/step - loss: 80.3189 - accuracy: 0.4771 - val_loss: 1.0991 - val_accuracy: 0.4648
Epoch 2/10
20/20 [==============================] - 253s 13s/step - loss: 1.0946 - accuracy: 0.4645 - val_loss: 1.0867 - val_accuracy: 0.4648
Epoch 3/10
20/20 [==============================] - 254s 13s/step - loss: 1.3666 - accuracy: 0.4613 - val_loss: 1.0687 - val_accuracy: 0.4648
Epoch 4/10
20/20 [==============================] - 256s 13s/step - loss: 2.5918 - accuracy: 0.4376 - val_loss: 1.0509 - val_accuracy: 0.4648
Epoch 5/10
20/20 [==============================] - 257s 13s/step - loss: 1.0450 - accuracy: 0.4645 - val_loss: 1.0354 - val_accuracy: 0.4648
Epoch 6/10
20/20 [==============================] - 256s 13s/step - loss: 1.0307 - accuracy: 0.4645 - val_loss: 1.0215 - val_accuracy: 0.4648
Epoch 7/10
20/20 [==============================] - 257s 13s/step - loss: 1.0177 - accuracy: 0.4645 - val_loss: 1.0086 - val_accuracy: 0.4648
Epoch

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


6/6 [==============================] - 55s 9s/step - loss: 0.9396 - accuracy: 0.4545
Epoch 1/5
19/19 [==============================] - 99s 4s/step - loss: 0.9220 - accuracy: 0.5160 - val_loss: 0.9732 - val_accuracy: 0.4894
Epoch 2/5
19/19 [==============================] - 80s 4s/step - loss: 0.9134 - accuracy: 0.5160 - val_loss: 0.9692 - val_accuracy: 0.4894
Epoch 3/5
19/19 [==============================] - 80s 4s/step - loss: 0.9061 - accuracy: 0.5160 - val_loss: 0.9658 - val_accuracy: 0.4894
Epoch 4/5
19/19 [==============================] - 80s 4s/step - loss: 0.8992 - accuracy: 0.5160 - val_loss: 0.9631 - val_accuracy: 0.4894
Epoch 5/5
6/6 [==============================] - 54s 9s/step - loss: 0.9111 - accuracy: 0.4545
Epoch 1/5
19/19 [==============================] - 104s 4s/step - loss: 0.9399 - accuracy: 0.4734 - val_loss: 0.8885 - val_accuracy: 0.5106
Epoch 2/5
19/19 [==============================] - 78s 4s/step - loss: 0.9373 - accuracy: 0.4734 - val_loss: 0.8856 - val_ac